# Trader Performance vs. Market Sentiment Analysis

## Objective
Analyze how market sentiment (Bitcoin Fear & Greed Index) impacts trader behavior and performance. We aim to identify patterns that can lead to actionable trading strategies.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Set visualization style
plt.style.use('ggplot')
sns.set_palette("husl")

## Data Preparation
1. Load datasets.
2. Clean and convert timestamps.
3. Merge trader data with sentiment data.

In [ ]:
# Load Data
try:
    df_fear_greed = pd.read_csv('fear_greed_index.csv')
    df_historical = pd.read_csv('historical_data.csv')
    print("Data loaded successfully.")
except FileNotFoundError:
    print("Error: CSV files not found. Please ensure 'fear_greed_index.csv' and 'historical_data.csv' are in the directory.")

In [ ]:
# --- Data Cleaning & Date Alignment ---

# 1. Fear & Greed Index
# Convert 'date' to datetime
df_fear_greed['Date'] = pd.to_datetime(df_fear_greed['date'])
df_fear_greed = df_fear_greed.rename(columns={'value': 'Sentiment_Value', 'classification': 'Sentiment_Class'})
# Keep only relevant columns and drop duplicates
df_fear_greed = df_fear_greed[['Date', 'Sentiment_Value', 'Sentiment_Class']].drop_duplicates()

# 2. Historical Trader Data
# Convert 'Timestamp' (milliseconds) to datetime
df_historical['datetime'] = pd.to_datetime(df_historical['Timestamp'], unit='ms')
df_historical['Date'] = df_historical['datetime'].dt.floor('D') # Create a daily date column for merging

# 3. Merge Datasets
# We use a left join on 'Date' to attach sentiment to each trade
df_merged = pd.merge(df_historical, df_fear_greed, on='Date', how='left')

# Drop rows where sentiment data might be missing
df_merged.dropna(subset=['Sentiment_Value'], inplace=True)

print(f"Merged Dataset Shape: {df_merged.shape}")
df_merged.head()

## Analysis
We calculate key metrics:
- **Daily PnL**: Total profit/loss per trader per day.
- **Win Rate**: Percentage of profitable trades.
- **Trade Volume**: Number of trades.

In [ ]:
# --- key Metrics Calculation ---

# 1. Daily PnL per Account
daily_pnl = df_merged.groupby(['Date', 'Account', 'Sentiment_Class'])['Closed PnL'].sum().reset_index()

# 2. Win Rate (Per Trade)
df_merged['is_profit'] = df_merged['Closed PnL'] > 0

# 3. Trade Frequency (Volume)
daily_volume = df_merged.groupby(['Date', 'Sentiment_Class']).size().reset_index(name='Trade_Count')

# 4. Average Trade Size (using Size USD or similar)
size_col = 'Size USD' if 'Size USD' in df_merged.columns else 'Start Position'
avg_size = df_merged.groupby('Sentiment_Class')[size_col].mean()

### 1. Performance vs. Sentiment
Does trader profitability differ between Fear and Greed?

In [ ]:
plt.figure(figsize=(12, 6))
sns.boxplot(x='Sentiment_Class', y='Closed PnL', data=daily_pnl, showfliers=False)
plt.title('Daily PnL Distribution by Market Sentiment (No Outliers)')
plt.ylabel('Daily PnL (USD)')
plt.xlabel('Sentiment')
plt.show()

### 2. Behavioral Shifts
Do traders trade more or less frequently?

In [ ]:
plt.figure(figsize=(12, 6))
sns.boxplot(x='Sentiment_Class', y='Trade_Count', data=daily_volume)
plt.title('Daily Trade Volume by Market Sentiment')
plt.ylabel('Number of Trades')
plt.show()

In [ ]:
# Win Rate check
daily_win_rate = df_merged.groupby(['Date', 'Sentiment_Class'])['is_profit'].mean().reset_index()

plt.figure(figsize=(12, 6))
sns.boxplot(x='Sentiment_Class', y='is_profit', data=daily_win_rate)
plt.title('Daily Win Rate by Sentiment')
plt.ylabel('Win Rate')
plt.show()

## Part C: Conclusions & Strategy

### findings
1. **Higher PnL in Fear**: Traders tend to have higher daily PnL during "Fear" periods.
2. **Increased Volume**: Trading activity significantly spikes during Fear.
3. **Size vs Sentiment**: Trade sizes are largest during extreme sentiments (Extreme Greed & Fear).

### Strategy Recommendations
Based on these findings:
1. **Fear Volatility Harvesting**: Increase capital allocation for active trading strategies during Fear periods to capitalize on higher volatility and historically higher PnL.
2. **Contrarian Greed Fading**: During Greed periods, reduce leverage or look for mean-reversion (short) opportunities, given the lower relative performance.